In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    nearest_report_date
)
import pandas as pd

In [2]:
# 获取回溯的报告日

today = dt.datetime.today().strftime("%Y%m%d")

In [8]:
# 获取所有目标QDII基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 6 AND
    (ENDDATE>='{today}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()

In [9]:
basic_info = fd_basicinfo(security_ids)
type_info = fd_typeclass(security_ids)

In [10]:
df = pd.merge(type_info, basic_info, on="SECURITYID")

In [14]:
df.L3NAME.unique()

array(['QDII股票型基金', 'QDII混合基金', 'QDII全球股票型基金', 'QDII债券基金', 'QDII指数股票型基金',
       'QDII商品基金', 'QDII房地产信托基金', 'QDII分级子基金'], dtype=object)

In [18]:
# 分类
df.loc[(df["L3NAME"].isin(["QDII股票型基金", "QDII全球股票型基金", "QDII指数股票型基金"])), "QDII型（子类）"] = "QDII股票型基金"
df.loc[df["L3NAME"] == "QDII债券基金", "QDII型（子类）"] = "QDII债券型基金"
df.loc[(df["L3NAME"].isin(["QDII商品基金", "QDII房地产信托基金"])), "QDII型（子类）"] = "QDII另类投资基金"
df.loc[(df["L3NAME"].isin(["QDII混合基金", "QDII分级子基金"])), "QDII型（子类）"] = "QDII混合型基金"

In [19]:
df.groupby("QDII型（子类）").count()

,SECURITYID,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
QDII型（子类）,,,,,,,,,,,,
QDII债券型基金,39,39,39,39,39,39,39,39,38,39,39,39
QDII另类投资基金,19,19,19,19,19,19,19,19,9,19,19,19
QDII混合型基金,31,31,31,31,31,31,31,31,13,31,31,31
QDII股票型基金,87,87,87,87,87,87,87,87,55,87,87,87
